In [43]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

def fetch_jobs():
    url = "https://www.mourjan.com/ae/al-ain/vacancies/en/?sl=1"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    jobs = []
    postings = soup.find_all("div", class_="ad")
    for post in postings:
        link_element = post.find("a", class_="link")
        date_element = post.find("div", class_="box hint")
        
        if link_element and date_element:
            job_link = link_element["href"].strip()
            date_divs = date_element.find_all("div")
            if len(date_divs) > 1:
                date_posted = date_divs[1].text.strip()
                
                job_title = post.find("div", {"dir": "auto"}).text.strip() if post.find("div", {"dir": "auto"}) else "No title"
                
                if "hour" in date_posted or "minute" in date_posted:
                    time_ago = int(date_posted.split()[0])
                    if "hour" in date_posted and time_ago <= 2:
                        jobs.append({"title": job_title, "link": f"https://www.mourjan.com{job_link}", "date": date_posted})
                    elif "minute" in date_posted:
                        jobs.append({"title": job_title, "link": f"https://www.mourjan.com{job_link}", "date": date_posted})

    return jobs

'''def send_whatsapp_message(jobs, contact_name):
    options = webdriver.ChromeOptions()
    options.add_argument('--user-data-dir=./User_Data')
    driver = webdriver.Chrome(options=options)

    driver.get("https://web.whatsapp.com")
    #time.sleep(15)  # Wait for manual QR code scan

    search_box = driver.find_element_by_xpath('//div[@contenteditable="true"][@data-tab="3"]')
    search_box.send_keys(contact_name)
    search_box.send_keys(Keys.ENTER)
    time.sleep(2)

    message_box = driver.find_element_by_xpath('//div[@contenteditable="true"][@data-tab="1"]')

    message = "New jobs posted within the last 2 hours:\n\n"
    for job in jobs:
        message += f"Title: {job['title']}\nLink: {job['link']}\nDate: {job['date']}\n\n"

    message_box.send_keys(message)
    message_box.send_keys(Keys.ENTER)
    time.sleep(2)
    driver.quit()'''

def main():
    new_jobs = fetch_jobs()
    if new_jobs:
        #send_whatsapp_message(new_jobs, "Suha")
        print("New jobs posted within the last 24 hours:")
        for job in new_jobs:
            print(f"Title: {job['title']}, Link: {job['link']}, Date: {job['date']}")
    else:
        print("No new jobs posted within the last 2 hours.")

if __name__ == "__main__":
    main()


New jobs posted within the last 24 hours:
Title: مطلوب بنت مصريه للعمل في كيوسك عطور في الهيلي مول خبرة بالبيع ولديه اقامة الدوام 8 ساعات ويوم اجازة بالأسبوع أرسل CV علي على الرقم ممنوع الاتصال واتساب وفقط, Link: https://www.mourjan.com/ae/al-ain/sales-and-marketing/vacancies/en/24953564/, Date: 32 minutes ago
Title: مطلوب على وجه السرعة مساعدة ادارية للعمل في منزل بمدينة العين - منطقة غمض، لديها خبرة في بنفس المجال ونشطة وفطنة البريد الألكتروني, Link: https://www.mourjan.com/ae/al-ain/administration/vacancies/en/24975389/, Date: 1 hour ago
Title: مطلوب فورا فورمان يكون لدية خبرة في تصنيع مظلات السيارات والبرجولات من الحديد والألومينيوم في الامارات ولدية خبرة في التسعير ويتوفر لديه عملاء مقر العمل العين منطقة عشارج براتب مقطوع ويكون لدية اقامة سارية, Link: https://www.mourjan.com/ae/al-ain/craftsmen/vacancies/en/24974948/, Date: 1 hour ago
Title: مطلوب بنت تجيد التساريح والخيط والبروتين الراتب 3000 والصالون في العين, Link: https://www.mourjan.com/ae/al-ain/beauty-care/vacancies/en/2497

In [19]:
pip install selenium webdriver-manager



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [41]:
#This is the whatsapp message sending workng

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pickle
import time

def save_session():
    options = webdriver.ChromeOptions()
    options.add_argument("user-data-dir=selenium")  # Path to user data
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get("https://web.whatsapp.com")

    print("Please scan the QR code with your phone to log in.")
    time.sleep(30)  # Increase the time if needed

    # Save cookies
    with open("whatsapp_cookies.pkl", "wb") as file:
        pickle.dump(driver.get_cookies(), file)

    driver.quit()

def load_session():
    options = webdriver.ChromeOptions()
    options.add_argument("user-data-dir=selenium")  # Path to user data
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get("https://web.whatsapp.com")

    # Load cookies
    with open("whatsapp_cookies.pkl", "rb") as file:
        cookies = pickle.load(file)
        for cookie in cookies:
            driver.add_cookie(cookie)

    driver.refresh()
    return driver

def send_whatsapp_message(contact_name, message_text):
    driver = load_session()

    try:
        # Wait for the page to load
        time.sleep(15)

        # Search for the contact
        search_box = driver.find_element(By.XPATH, '//div[@contenteditable="true"][@role="textbox"][@aria-label="Search input textbox"]')
        search_box.click()
        search_box.send_keys(contact_name)
        search_box.send_keys(Keys.ENTER)

        # Wait for the contact to open
        time.sleep(2)

        # Find the message box and send the message
        message_box = driver.find_element(By.XPATH, '//div[@contenteditable="true"][@role="textbox"][@aria-label="Type a message"]')
        message_box.click()
        message_box.send_keys(message_text)
        message_box.send_keys(Keys.ENTER)

        print("Message sent successfully!")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        time.sleep(5)
        driver.quit()

# First, save the session by scanning the QR code once
save_session()

# Then, use the session to send messages
contact_name = 'suha'  # Replace with the contact's name
message_text = 'Hello, this is an automated message from Selenium!'
send_whatsapp_message(contact_name, message_text)


Please scan the QR code with your phone to log in.
Message sent successfully!


In [50]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle
import time

def fetch_jobs():
    url = "https://www.mourjan.com/ae/al-ain/vacancies/en/?sl=1"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    jobs = []
    postings = soup.find_all("div", class_="ad")
    for post in postings:
        link_element = post.find("a", class_="link")
        date_element = post.find("div", class_="box hint")
        
        if link_element and date_element:
            job_link = link_element["href"].strip()
            date_divs = date_element.find_all("div")
            if len(date_divs) > 1:
                date_posted = date_divs[1].text.strip()
                
                job_title = post.find("div", {"dir": "auto"}).text.strip() if post.find("div", {"dir": "auto"}) else "No title"
                
                if "hour" in date_posted or "minute" in date_posted:
                    time_ago = int(date_posted.split()[0])
                    if "hour" in date_posted and time_ago <= 24:
                        jobs.append({"title": job_title, "link": f"https://www.mourjan.com{job_link}", "date": date_posted})
                    elif "minute" in date_posted:
                        jobs.append({"title": job_title, "link": f"https://www.mourjan.com{job_link}", "date": date_posted})

    return jobs

def save_session():
    options = webdriver.ChromeOptions()
    options.add_argument("user-data-dir=selenium")  # Path to user data
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get("https://web.whatsapp.com")

    print("Please scan the QR code with your phone to log in.")
    time.sleep(60)  # Increase the time if needed

    # Save cookies
    with open("whatsapp_cookies.pkl", "wb") as file:
        pickle.dump(driver.get_cookies(), file)

    driver.quit()

def load_session(driver):
    # Load cookies
    with open("whatsapp_cookies.pkl", "rb") as file:
        cookies = pickle.load(file)
        for cookie in cookies:
            driver.add_cookie(cookie)
    
    driver.refresh()

def send_whatsapp_message(jobs, contact_name):
    options = webdriver.ChromeOptions()
    options.add_argument("user-data-dir=selenium")  # Path to user data
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get("https://web.whatsapp.com")

    # Load session data
    load_session(driver)

    # Refresh the page to apply the session data
    driver.refresh()

    try:
        # Wait for the page to load
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, '//div[@contenteditable="true"][@role="textbox"][@aria-label="Search input textbox"]'))
        )

        # Search for the contact
        search_box = driver.find_element(By.XPATH, '//div[@contenteditable="true"][@role="textbox"][@aria-label="Search input textbox"]')
        search_box.click()
        search_box.send_keys(contact_name)
        search_box.send_keys(Keys.ENTER)

        # Wait for the contact to open
        time.sleep(2)

        # Find the message box and send the message
        message_box = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//div[@contenteditable="true"][@role="textbox"][@aria-label="Type a message"]'))
        )
        message_box.click()

        # Create message text
        message = "New jobs posted within the last 2 hours:\n\n"
        for job in jobs:
            message += f"Title: {job['title']}\nLink: {job['link']}\nDate: {job['date']}\n\n"

        # Send the message
        message_box.send_keys(message)
        message_box.send_keys(Keys.ENTER)

        print("Message sent successfully!")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        time.sleep(5)
        driver.quit()

def main():
    new_jobs = fetch_jobs()
    if new_jobs:
        send_whatsapp_message(new_jobs, "FriendName")
    else:
        print("No new jobs posted within the last 24 hours.")

if __name__ == "__main__":
    # First, save the session by scanning the QR code once
    #save_session()

    # Then, use the session to send messages
    main()


Message sent successfully!
